# Regression for parkingMiddleGoalsDemo

In [1]:
#!/usr/bin/env python
# coding: utf-8

import sys
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math


# assign abs. path value 

car_path = r"D:\Z_DesayWorkSpace\5023_path_jinbiao\5023_20220402\4_pc/"

# for parkingMiddleGoalsDemo
# log_car_file = car_path+"parkingMiddleGoalsDemo.log"
# path_car_file = car_path+"parkingMiddleGoalsDemo_reg.txt"

# for parkingMiddleGoals_original
# log_car_file = car_path+"parkingMiddleGoals_original.log"
# path_car_file = car_path+"parkingMiddleGoals_original_reg.txt"


# for ros_parkingMiddleGoalsDemo
# log_car_file = car_path+"ros_parkingMiddleGoalsDemo.log"
# path_car_file = car_path+"ros_parkingMiddleGoalsDemo.txt"

log_car_file = car_path+"logfile_all.txt"
path_car_file = car_path+"computed_path.txt"


# # for ros_parkingMiddleGoals_original
# log_car_file = car_path+"ros_parkingMiddleGoals_original.log"
# path_car_file = car_path+"ros_parkingMiddleGoals_original.txt"


# # # for ros_parkingMiddleGoals_original_ryan
# log_car_file = car_path+"ros_parkingMiddleGoals_original_ryan.log"
# path_car_file = car_path+"ros_parkingMiddleGoals_original_ryan.txt"


print("log path: ", log_car_file)
print("car path: ", path_car_file)

log path:  D:\Z_DesayWorkSpace\5023_path_jinbiao\5023_20220402\4_pc/logfile_all.txt
car path:  D:\Z_DesayWorkSpace\5023_path_jinbiao\5023_20220402\4_pc/computed_path.txt


In [2]:
# parse log file: parkingMiddleGoalsDemo_reg.log

shared_state_goal_x = []
shared_state_goal_y = []
shared_state_goal_theta = []

true_goal_x = []
true_goal_y = []
true_goal_theta = []

correct_goal_x = []
correct_goal_y = []
correct_goal_theta = []

odom_goal_x = []
odom_goal_y = []
odom_goal_theta = []

with (open(log_car_file,"r")) as f:
    begin_t = 0.0
    time_set = False
    current_t = 0
    control_begin = False
    get_cache = False
    frame_start = 0
    frame_start_apa = 0
    frame_end = 0
    odom_start = 0
    
    current_log_time = 0
    current_log_time1 = 0
    
    line_num = 0
    
    start_goal = False
    goal_reached = False
    
    
    for line in f:

        line_num += 1
        
        # remove "[tsl]" in line
        if "[tsl]" in line:
            line = line.replace("[tsl]", "")
            
        # check if goal reached -- 13
        elif ("The goal is reached" in line):
            print("\n13. The goal is reached ok")
            goal_reached = True
        
        
        # get true goal from shared state - 15
        elif "ParkingMiddleGoals: received goal from sharedstate" in line:
            print ("\n1. ParkingMiddleGoals: received goal from sharedstate ok")
            print(line.split())
            raw_shared_state_x = line.split()[8]
            raw_shared_state_y = line.split()[9]
            raw_shared_state_theta = line.split()[10]
            shared_state_goal_x.append(float(raw_shared_state_x))
            shared_state_goal_y.append(float(raw_shared_state_y))
            shared_state_goal_theta.append(float(raw_shared_state_theta))
        
        
        # get true goal
        elif "getPerpendicularPath: goal point" in line:
            print("\n2. getPerpendicularPath: goal point  ok")
            print(line.split())
            raw_true_x = line.split()[6]
            raw_true_y = line.split()[7]            
            raw_true_theta = line.split()[8]   
            true_goal_x.append(float(raw_true_x))
            true_goal_y.append(float(raw_true_y))
            true_goal_theta.append(float(raw_true_theta))
            
        
        # get corrected goal point for origianl
        elif "getPerpendicularPath: after correction goal point" in line:
            try:
                print("\n3. getPerpendicularPath: after correction goal point ok")
                print(line.split())
                raw_correct_x = line.split()[8]
                raw_correct_y = line.split()[9]            
                raw_correct_theta = line.split()[10]   
                correct_goal_x.append(float(raw_correct_x))
                correct_goal_y.append(float(raw_correct_y))
                correct_goal_theta.append(float(raw_correct_theta))
            except:
                print("Unexpected error original:", sys.exc_info()[0])
                raise

                
        
        # get corrected goal point for demo
        elif "ParkingMiddleGoalsDemo: corrected final goal" in line:
            try:
                print("\n4. ParkingMiddleGoalsDemo: corrected final goal ok")
                print(line.split())
                raw_correct_x = line.split()[10]
                raw_correct_y = line.split()[11]            
                raw_correct_theta = line.split()[12]   
                correct_goal_x.append(float(raw_correct_x))
                correct_goal_y.append(float(raw_correct_y))
                correct_goal_theta.append(float(raw_correct_theta))
            except:
                print("Unexpected error demo:", sys.exc_info()[0])
                raise

                
        # get corrected goal point for ryan
                
        # check if goal reached -- 13
        elif ("The goal is reached" in line):
            print("\n13. The goal is reached ok")
            print(line.split())
            goal_reached = True
        
        # parse odometry -- 2
        if ("Current Pose(x, y, theta, phi):" in line):
#             print("\nCurrent Pose(x, y, theta, phi): ok")
#             print(line.split())
            try:
                raw_odom_goal_x = line.split()[8]
                raw_odom_goal_x = raw_odom_goal_x.replace('(','').replace(',','')
                raw_odom_goal_y = line.split()[9]
                raw_odom_goal_y = raw_odom_goal_y.replace(',','').replace(')','')
                raw_odom_goal_theta = line.split()[10]
                raw_odom_goal_theta = raw_odom_goal_theta.replace(',','')
                raw_odom_goal_phi = line.split()[11]
                raw_odom_goal_phi = raw_odom_goal_phi.replace(')','')
                if (goal_reached):
                    odom_goal_x.append(float(raw_odom_goal_x))
                    odom_goal_y.append(float(raw_odom_goal_y))
                    odom_goal_theta.append(float(raw_odom_goal_theta))
                    goal_reached = False
            except:
                print("Unexpected error demo:", sys.exc_info()[0])
                raise
            pass

        
        
#         elif "ParkingMiddleGoals: 3 m_targetpose" in line:
#             try:
#                 print("\n5. ParkingMiddleGoals: 3 m_targetpose ok")
#                 print(line.split())
#                 raw_correct_x = line.split()[9]
#                 raw_correct_y = line.split()[10]            
#                 raw_correct_theta = line.split()[11]   
#                 correct_goal_x.append(float(raw_correct_x))
#                 correct_goal_y.append(float(raw_correct_y))
#                 correct_goal_theta.append(float(raw_correct_theta))
#             except:
#                 print("Unexpected error demo:", sys.exc_info()[0])
#                 raise
                

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Z_DesayWorkSpace\\5023_path_jinbiao\\5023_20220402\\4_pc/logfile_all.txt'

In [ ]:
# parse path file: parkingMiddleGoalsDemo_reg.txt

# load computed path files
try:
    path_carx = []
    path_cary = []

    with (open(path_car_file,"r")) as f:
        for line in f:
            if ("x y" not in line):
                path_carx.append(float(line.split()[0]))
                path_cary.append(float(line.split()[1]))
except:
    print("parkingMiddleGOalsDemo_reg path error")     

In [ ]:
# plot true goal, corrected goal, planned path

plt.figure(figsize=(8,10))

# 1. plot computed path
plt.plot(path_carx, path_cary, label="car_path1", linewidth=6, alpha=0.5)

# 9: plot shared state goal
plt.scatter(shared_state_goal_x, shared_state_goal_y, label="shared_state_goal", marker='+', color='r', s=300)
print("shared goal [x, y, theta]:\t", shared_state_goal_x, shared_state_goal_y, shared_state_goal_theta)

# 7. plot true goal
plt.scatter(true_goal_x, true_goal_y, label="true_goal", marker='D', color='g', s=200, alpha=0.6)
print("true goal [x, y, theta]:\t", true_goal_x, true_goal_y, true_goal_theta)

# 8. plot correct goal
plt.scatter(correct_goal_x, correct_goal_y, label="correct_goal", marker='*', color='m', s=300, alpha=0.6)
print("correct goal [x, y, theta]:\t", correct_goal_x, correct_goal_y, correct_goal_theta)

# 9. plot odom goal
plt.scatter(odom_goal_x, odom_goal_y, label="odom_goal", marker='*', color='k', s=150)
print("odom goal [x, y, theta]:\t", odom_goal_x, odom_goal_y, odom_goal_theta)


plt.legend(loc="best")

plt.show()
